In [1]:
import numpy as np
import pandas as pd

In [2]:
chars = ' !"\'(),-.0123456789:;?abcdefghijklmnopqrstuvwxyz'
chars_list = [*chars]
print(len(chars_list))

48


In [3]:
# Interpolate probabilities of each characters
# Lambdas equaly likely

def interpolation(probs_dic):
    chars_probs = {}
    for i in probs_dic.keys():
        if len(probs_dic[i]) > 0:
            char_prob = (1/len(probs_dic[i]))*sum(list(probs_dic[i].values()))
            chars_probs[i] = char_prob
        else:
            chars_probs[i] = 0
    return chars_probs

In [4]:
# Interpolate probabilities of each characters
# Lambdas uneven
# I have tried many combinations and this one had the best result
# Although its not optimum for sure

def interpolation2(probs_dic):
  lambda5 = 0.95
  lambda4 = 0.03
  lambda3 = 0.01
  lambda2 = 0.005
  lambda1 = 0.005
  chars_probs = {}
  for i in probs_dic.keys():
    chars_probs[i] = lambda5*probs_dic[i][5] + lambda4*probs_dic[i][4] + lambda3*probs_dic[i][3] + lambda2*probs_dic[i][2] + lambda1*probs_dic[i][1]
  return chars_probs

In [5]:
# Normalize interpolated probabilities of each character; sum to 1

def softmax(chars_probs):
    probs_sum = sum(list(chars_probs.values()))
    return {key: value / probs_sum for key, value in chars_probs.items()}

In [15]:
# input: gram_degree, vocabulary (48 ASCII characters as string), train and test data
# output: loss

def from_scratch(gram_degree, chars_list, train, test):
    start = 0
    end = gram_degree
    loss = 0
    counter = 0
    len_train = len(train)
    for i in range(len(test) - end + 1):
        pattern = test[start:end]
        start += 1
        end += 1
        counter += 1
        ch_probs_dic = {}
        for j in range(len(chars_list)):
            ch = chars_list[j]
            token = pattern[:-1] + ch
            len_token = len(token)
            probs_dict = {}
            for k in range(len(token)):
                c = train.count(token[k:])
                if k < len_token - 1:
                    prob = (c+1) / (train.count(token[k:-1])+1)
                if k == len_token - 1:
                    prob = (c+1) / (len_train+48)
                probs_dict[len_token-k] = prob
            ch_probs_dic[ch] = probs_dict
        ch_probs_sfx = softmax(interpolation2(ch_probs_dic))
        loss -= np.log2(ch_probs_sfx[pattern[-1]])
    return loss/counter        

In [41]:
# input: gram_degree, vocabulary (48 ASCII characters as string), train and test data
# output: loss
# without interpolation

def from_scratch_2(gram_degree, chars_list, train, test):
    start = 0
    end = gram_degree
    loss = 0
    counter = 0
    len_train = len(train)
    for i in range(len(test) - end + 1):
        pattern = test[start:end]
        start += 1
        end += 1
        counter += 1
        ch_probs_dic = {}
        for j in range(len(chars_list)):
            ch = chars_list[j]
            token = pattern[:-1] + ch
            len_token = len(token)
            probs_dict = {}
            for k in range(len_token):
                c = train.count(token[k:])
                if (c>0) and (k < len_token - 1):
                    prob = c / train.count(token[k:-1])
                    break
                elif (c>0) and (k == len_token - 1):
                    prob = c / len_train
                else:
                    pass
            ch_probs_dic[ch] = prob
        ch_probs_sfx = softmax(ch_probs_dic)
        loss -= np.log2(ch_probs_sfx[pattern[-1]])
    return loss/counter     

In [6]:
with open(r"D:\SLU\AI MSc\Fall 22\NLP\Project 4\cwe-train.txt", 'r') as f:
    train = f.readlines()
with open(r"D:\SLU\AI MSc\Fall 22\NLP\Project 4\cwe-test.txt", 'r') as f:
    test = f.readlines()

In [7]:
train = train[0]
test = test[0]
print(len(train))
print(len(test))

603432
61717


In [39]:
from_scratch(5, chars_list, train, test[:30])

1.5352314885678935

In [42]:
# without interpolation
# faster

from_scratch_2(5, chars_list, train, test[:30])

2.087423659118948